In [ ]:
def get_data(pokemon_name_list):
    soup_list = []

    for pokemon_name in pokemon_name_list:
        base_url = f'https://www.ebay.fr/sch/i.html?_from=R40&_nkw={pokemon_name}&_sacat=0&LH_Sold=1&LH_Complete=1&rt=nc&LH_PrefLoc=1&_ipg=100'
        
        # Récupérer le contenu de la première page
        r = requests.get(base_url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            soup_list.append(soup)

            # Vérifier s'il y a plus d'une page
            pagination_items = soup.find('ol', {'class': 'pagination__items'})
            if pagination_items:
                # Récupérer le nombre total de pages
                num_pages = len(pagination_items.find_all('li'))

                # Récupérer les données des pages suivantes
                for page in range(2, num_pages + 1):
                    next_url = f'{base_url}&_pg={page}'
                    next_r = requests.get(next_url)
                    if next_r.status_code == 200:
                        next_soup = BeautifulSoup(next_r.text, 'html.parser')
                        soup_list.append(next_soup)
                    else:
                        print(f"La requête pour {pokemon_name} (page {page}) a échoué avec le code d'état {next_r.status_code}")

                    # Attendre 20 secondes avant la prochaine requête
                    time.sleep(20)
        else:
            print(f"La requête pour {pokemon_name} a échoué avec le code d'état {r.status_code}")

    return soup_list

In [ ]:
def get_data2(pokemon_name_list):
    soup_list = []

    for pokemon_name in pokemon_name_list:
        base_url = f'https://www.ebay.fr/sch/i.html?_from=R40&_nkw={pokemon_name}&_sacat=0&LH_Sold=1&LH_Complete=1&rt=nc&LH_PrefLoc=1&_ipg=100'
        
        # Récupérer le contenu de la première page
        r = requests.get(base_url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            soup_string =soup.prettify()
        
        
        index1 = soup_string.find("srp-results srp-list clearfix")
        if index1 != -1:
            soup_string =  soup_string[index1:] 
        
        index2 = soup_string.find("à moins de mots")
        if index2 != -1:
            soup_string =  soup_string[:index2]
        
        soup = BeautifulSoup(soup_string, 'html.parser')
        soup_list.append(soup)
       
    return soup_list

In [ ]:
def parse(soup_list):
    cards_list = []

    for soup in soup_list:
        results = soup.find_all('div', {'class': "s-item__info clearfix"})
        for item in results:
            title_span = item.find('div', {'class': 's-item__title'})
            
            card_name = title_span.text if title_span else None
            
            # Trouver le span parent avec la classe 's-item__title--tag'
            title_tag_span = item.find('div', {'class': 's-item__title--tag'})
            
            # Vérifier si l'élément est trouvé avant d'essayer d'accéder à ses attributs
            if title_tag_span:
                # Trouver le span imbriqué avec la classe 'POSITIVE'
                sold_date_span = title_tag_span.find('span', {'class': 'POSITIVE'})
                sold_date = sold_date_span.text.replace('Vendu le', '') if sold_date_span else None
            else:
                sold_date = None
            
            # Initialiser location avec None avant la condition
            location = None

            detail_tag_div = item.find('div', {'class': 's-item__details clearfix'})
            
            if detail_tag_div:
                primary_location_tag_span = item.find('div', {'class': 's-item__detail s-item__detail--primary'})
                
                

                if primary_location_tag_span:
                    location_tag_span1 = item.find('span', {'class': 's-item__location s-item__itemLocation'})
                    
                    

                    if location_tag_span1:
                        location = location_tag_span1.find('span',{'class': 'ITALIC'})
                        location = location.text if location else None
            
            vendor_name_element = item.find('span', {'class': 's-item__seller-info-text'})
            vendor_name = vendor_name_element.text if vendor_name_element else None
            
            bids_numbers_element = item.find('span', {'class': 's-item__bids'})
            bids_numbers = bids_numbers_element.text.replace('\xa0enchères', '').replace('enchère','') if bids_numbers_element else None

            product = {
                'card_name': card_name,
                'card_price': item.find('span', {'class': 's-item__price'}).text.replace('€', '').replace(',', '.').strip(),
                'sold_date': sold_date,
                'vendor_name': vendor_name,
                'bids_numbers': bids_numbers,
                'sold_location' : location
            }
            cards_list.append(product)

    return cards_list

In [ ]:
def get_parse_2(pokemon_name_list):
    soup_list = []
    cards_list = []

    for pokemon_name in pokemon_name_list:
        start_time_pokemon = time.time()  # Temps de début du parsing pour ce Pokémon

        base_url = f'https://www.ebay.fr/sch/i.html?_from=R40&_nkw={pokemon_name}&_sacat=0&LH_Sold=1&LH_Complete=1&rt=nc&LH_PrefLoc=1&_ipg=100'
        
        # Récupérer le contenu de la première page
        r = requests.get(base_url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            soup_string =soup.prettify()
        
        
        index1 = soup_string.find("srp-results srp-list clearfix")
        if index1 != -1:
            soup_string =  soup_string[index1:] 
        
        index2 = soup_string.find("à moins de mots")
        if index2 != -1:
            soup_string =  soup_string[:index2]
        
        soup = BeautifulSoup(soup_string, 'html.parser')
        soup_list.append(soup)
    
        

        for soup in soup_list:
            results = soup.find_all('div', {'class': "s-item__info clearfix"})
            for item in results:
                title_span = item.find('div', {'class': 's-item__title'})
                
                card_name = title_span.text if title_span else None
                
                # Trouver le span parent avec la classe 's-item__title--tag'
                title_tag_span = item.find('div', {'class': 's-item__title--tag'})
                
                # Vérifier si l'élément est trouvé avant d'essayer d'accéder à ses attributs
                if title_tag_span:
                    # Trouver le span imbriqué avec la classe 'POSITIVE'
                    sold_date_span = title_tag_span.find('span', {'class': 'POSITIVE'})
                    sold_date = sold_date_span.text.replace('Vendu le', '') if sold_date_span else None
                else:
                    sold_date = None
                
                # Initialiser location avec None avant la condition
                location = None

                detail_tag_div = item.find('div', {'class': 's-item__details clearfix'})
                
                if detail_tag_div:
                    primary_location_tag_span = item.find('div', {'class': 's-item__detail s-item__detail--primary'})
                    
                    

                    if primary_location_tag_span:
                        location_tag_span1 = item.find('span', {'class': 's-item__location s-item__itemLocation'})
                        
                        

                        if location_tag_span1:
                            location = location_tag_span1.find('span',{'class': 'ITALIC'})
                            location = location.text if location else None
                
                vendor_name_element = item.find('span', {'class': 's-item__seller-info-text'})
                vendor_name = vendor_name_element.text if vendor_name_element else None
                
                bids_numbers_element = item.find('span', {'class': 's-item__bids'})
                bids_numbers = bids_numbers_element.text.replace('\xa0enchères', '').replace('enchère','') if bids_numbers_element else None

                product = {
                    'card_name': card_name,
                    'card_price': item.find('span', {'class': 's-item__price'}).text.replace('€', '').replace(',', '.').strip(),
                    'sold_date': sold_date,
                    'vendor_name': vendor_name,
                    'bids_numbers': bids_numbers,
                    'sold_location' : location
                }
                cards_list.append(product)

        end_time_pokemon = time.time()  # Temps de fin du parsing pour ce Pokémon
        elapsed_time_pokemon = end_time_pokemon - start_time_pokemon  # Temps total pour ce Pokémon
        print(f"Le parsing pour le Pokémon {pokemon_name} a pris {elapsed_time_pokemon} secondes")

    return cards_list

In [ ]:
def card_name_formatting(df : pd.DataFrame):

    df['card_name'] = df['card_name'].apply(unidecode)
    df['card_name'] = df['card_name'].str.lower()
    
    
    
    for row_index, row in df.iterrows():
        for pokemon_split_name in pokemon_list:
            conditions = all(part in row['card_name'] for part in pokemon_split_name)
            additional_conditions = ('carte' in row['card_name']) or ('fr' in row['card_name'])
            if conditions and additional_conditions:
                df.at[row_index, 'card_name'] = ' '.join(pokemon_split_name)
    
    df = df[df['card_name'].isin([' '.join(parts) for parts in pokemon_list])]
    df = df.reset_index()
    df = df.drop(columns='index')

    return df

In [ ]:
def transform(df : pd.DataFrame):
    
    # On supprime la première ligne car elle contient de mauvaises infos
    if not df.empty:
        df = df.dropna(subset=['sold_date'])
        df = df.reset_index(drop=True)


    
    '''cards_df['vendor_name'] = cards_df['vendor_name'].str.replace(r'\s*\(\d+\)\s*|\s*\d+%?$', '', regex=True)
    cards_df['vendor_name'] = cards_df['vendor_name'].str.replace(r'\s*\(\d+\)\s*', '',regex=True)
    cards_df['vendor_name'] = cards_df['vendor_name'].str.replace(r'\s*\(\d+\)\s*\d*$', '', regex=True)'''

    # EDIT NE pas drop la colonne simplement séparé la colonne en 3 colonnes par les espaces puis ne garder que la 1 èere colonne du split qui contient le nom du vendeur
    # df.drop(columns='vendor_name',inplace=True)
    #df['vendor_name'] = df['vendor_name'].str.split(expand=True)

    df['vendor_name'] = df['vendor_name'].str.replace(r'\s+', '', regex=True)
    df['vendor_name'] = df['vendor_name'].replace(r'\((\d+)\)', r' (\1) ', regex=True)
    df[['vendor_name', 'vendor_reviews', 'vendor_ratings']] = df['vendor_name'].str.split(' ', expand=True) 
    df = df.dropna(subset=['vendor_ratings'])

    
    
    #On supprime les parenthèses de la colonne 'vendor_reviews'
    df['vendor_reviews'] = df['vendor_reviews'].str.replace('(', '')
    df['vendor_reviews'] = df['vendor_reviews'].str.replace(')', '').astype(int)

    # Supprimer le pourcentage de la colonne 'vendor_ratings'
    df['vendor_ratings'] = df['vendor_ratings'].str.replace('%', '').replace(',','.')
    df['vendor_ratings'] = df['vendor_ratings'].str.replace(',','.').astype(float)
    
    
    # Drop & Renommage de la colonne 'bids_numbers' pour garder une cohérence d'odre des colonnes
    df['bids_numbers2'] = df['bids_numbers']
    df.drop(columns='bids_numbers',inplace=True)
    df.rename(columns={'bids_numbers2':'bids_numbers'},inplace=True)
  

    # Renommez les colonnes résultantes
    df.rename(columns={0: 'vendor_name', 1: 'vendor_reviews', 2: 'vendor_ratings'}, inplace=True)

    # On supprime "EUR" puis le change le nom de la colonne pour spécifié la devise
    df['card_price'] = df['card_price'].str.replace('EUR','')
    df = df.rename(columns={'card_price': 'card_price_EUR'})


    df = card_name_formatting(df)

    # Conversion des colonnes 

    # Conversion df['card_price] en float

    df['card_price_EUR'] = pd.to_numeric(df['card_price_EUR'], errors='coerce')

    # Conversion df['sold_date'] en datetime64 
    
    # On supprime les accents de la colonne 'sold_date'
    df['sold_date'] = df['sold_date'].apply(unidecode)
    df['sold_date'] = df['sold_date'].str.strip()
    df['sold_date'] = df['sold_date'].astype(str)

    # Supprimer les points de la colonne 'sold_date'
    df['sold_date'] = df['sold_date'].str.replace('.', '')
    df['sold_date'] = df['sold_date'].str.replace('sept', 'sep')
    df['sold_date'] = df['sold_date'].str.replace('sept.', 'sep')
    df['sold_date'] = df['sold_date'].str.replace('janv', 'jan')
    df['sold_date'] = df['sold_date'].str.replace('fevr.', 'feb')
    df['sold_date'] = df['sold_date'].str.replace('fevr', 'feb')

    # Convertir la colonne 'sold_date' en type datetime si nécessaire
    df['sold_date'] = pd.to_datetime(df['sold_date'], format="%d %b %Y",errors='coerce')


    # Remplacement des "None" ie pas d'enchères par "0" puis conversion en int

    df['bids_numbers'] = df['bids_numbers'].fillna(0)
    df['bids_numbers'] = pd.to_numeric(df['bids_numbers'], errors='coerce')
    df['bids_numbers'] = df['bids_numbers'].fillna(0)

    df.dropna(inplace=True)
    
    # On retourne le df ainsi transformé
    return df